# 아마존 베드록 소개

> *이 노트북에서 사용하는 베드록 모델에 대한 허용 목록에 포함되어 있지 않다면 오류가 발생할 수 있습니다*

> *이 노트북은 세이지메이커 스튜디오의 **`Data Science 3.0`** 커널에서 잘 작동할 것입니다*

---

이 데모 노트북에서는 [`boto3` 파이썬 SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)를 활용해 [아마존 베드록](https://aws.amazon.com/bedrock/) 파운데이션 모델을 다루는 방법을 보여드리겠습니다.

---

In [2]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for boto3>=1.28.57 from https://files.pythonhosted.org/packages/45/78/e95de33d8850429f6740763eb2d5350c02fd39e916f2ddb69fe649d25125/boto3-1.29.6-py3-none-any.whl.metadata
  Obtaining dependency information for awscli>=1.29.57 from https://files.pythonhosted.org/packages/c9/fa/708f561f2ec441b9e9f6e9780a0519a7922a8e1f02c92671454983ff44c3/awscli-1.30.6-py3-none-any.whl.metadata
  Obtaining dependency information for botocore>=1.31.57 from https://files.pythonhosted.org/packages/e4/dd/a6c7154d4f045599ce2df9199c01a8c2650c45ae74ad63f69547a500cc1c/botocore-1.32.6-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8874a96c5061422625ba86c3ee1e6587ccd92ff9f5bf7bd91b2/s3transfer-0.7.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 80.2 MB/s eta 0:00:00
  Ob

이 노트북은 AWS SDK를 직접 사용하여 베드록 모델을 호출하는 방법을 보여줍니다. 하지만 이후 워크샵의 다른 노트북을 위해서는 [LangChain](https://github.com/hwchase17/langchain)도 설치해야 합니다.

In [3]:
%pip install --quiet langchain==0.0.309


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


---

## boto3 클라이언트 생성

베드록 API와의 상호 작용은 파이썬용 AWS SDK인 [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)를 통해 이루어집니다.

#### 서로 다른 클라이언트 사용
boto3는 아마존 베드록에 대해 서로 다른 작업을 수행하기 위한 여러 클라이언트를 제공합니다. [`InvokeModel`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html)과 [`InvokeModelWithResponseStream`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html) 작업은 아마존 베드록 런타임에서 지원되는 반면, [ListFoundationModels](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_ListFoundationModels.html)와 같은 다른 작업은 [아마존 베드록 클라이언트](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock.html)를 통해 처리됩니다.

#### 기본 자격 증명 체인 사용

이 노트북을 [아마존 세이지메이커 스튜디오](https://aws.amazon.com/sagemaker/studio/)에서 실행하고 있고 세이지메이커 스튜디오 [실행 역할](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)에 베드록 접근 권한이 있다면 아래 셀을 그대로 실행하면 됩니다. 베드록에 접근할 수 있는 기본 AWS 자격 증명이 설정된 컴퓨터에서 이 노트북을 실행하는 경우도 마찬가지입니다.

#### 다른 AWS 리전 사용

자체 컴퓨터나 베드록이 설정된 곳과 다른 AWS 리전의 세이지메이커 노트북에서 이 노트북을 실행하는 경우, 아래의 `os.environ['AWS_DEFAULT_REGION']` 줄의 주석을 해제하고 사용할 리전을 지정할 수 있습니다.

#### 특정 프로파일 사용

여러 프로파일로 AWS CLI를 설정한 자체 컴퓨터에서 이 노트북을 실행하고 있고, 베드록에 접근할 수 있는 프로파일이 기본 프로파일이 아닌 경우, 아래의 `os.environ['AWS_PROFILE']` 줄의 주석을 해제하고 사용할 프로파일을 지정할 수 있습니다.

#### 다른 역할 사용

귀하나 귀하의 회사가 베드록에 접근하기 위한 특정 별도의 [IAM 역할](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html)을 설정한 경우, 아래의 `os.environ['BEDROCK_ASSUME_ROLE']` 줄의 주석을 해제하여 지정할 수 있습니다. 현재 사용자나 역할이 해당 역할을 맡을 수 있는 [권한](https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRole.html)이 있는지 확인하세요.

#### `langchain`에 대한 참고 사항

`langchain`에서 제공하는 베드록 클래스는 기본적으로 베드록 boto3 클라이언트를 생성합니다. 베드록 구성을 사용자 지정하려면 아래 방법을 사용하여 베드록 클라이언트를 명시적으로 생성하고, `client=bedrock_runtime`을 사용하여 [`langchain.Bedrock`](https://python.langchain.com/docs/integrations/llms/bedrock) 클래스 인스턴스화 메서드에 전달하는 것을 권장합니다.

In [4]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

#### 연결 확인

`list_foundation_models()` 메서드를 사용해 클라이언트가 제대로 작동하는지 확인할 수 있습니다. 이 메서드는 우리가 사용할 수 있는 모든 모델을 알려줍니다.

In [5]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'dcda90a1-0842-4773-8305-454722d97092',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 25 Nov 2023 03:10:12 GMT',
   'content-type': 'application/json',
   'content-length': '9628',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'dcda90a1-0842-4773-8305-454722d97092'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND']},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-medium',
   'modelId': 'amazon.titan-tg1-medium',
   'modelName': 'Titan Text Medium',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'r

---

## `InvokeModel` 본문과 출력

아마존 베드록 런타임 클라이언트의 `invoke_model()` 메서드(`InvokeModel` API)는 어떤 모델을 사용하든 대부분의 텍스트 생성 및 처리 작업에 주로 사용하는 메서드가 될 것입니다.

메서드는 공유되지만, 입력과 출력의 형식은 사용하는 파운데이션 모델에 따라 다릅니다. 이는 아래에 설명된 바와 같습니다.

### Amazon Titan Large

#### Input
```json
{   
    "inputText": "<prompt>",
    "textGenerationConfig" : { 
        "maxTokenCount": 512,
        "stopSequences": [],
        "temperature": 0.1,  
        "topP": 0.9
    }
}
```

#### Output

```json
{
    "inputTextTokenCount": 613,
    "results": [{
        "tokenCount": 219,
        "outputText": "<output>"
    }]
}
```

### AI21 Jurassic (Grande and Jumbo) 

#### Input

```json
{
    "prompt": "<prompt>",
    "maxTokens": 200,
    "temperature": 0.5,
    "topP": 0.5,
    "stopSequences": [],
    "countPenalty": {"scale": 0},
    "presencePenalty": {"scale": 0},
    "frequencyPenalty": {"scale": 0}
}
```

#### Output

```json
{
    "id": 1234,
    "prompt": {
        "text": "<prompt>",
        "tokens": [
            {
                "generatedToken": {
                    "token": "\u2581who\u2581is",
                    "logprob": -12.980147361755371,
                    "raw_logprob": -12.980147361755371
                },
                "topTokens": null,
                "textRange": {"start": 0, "end": 6}
            },
            //...
        ]
    },
    "completions": [
        {
            "data": {
                "text": "<output>",
                "tokens": [
                    {
                        "generatedToken": {
                            "token": "<|newline|>",
                            "logprob": 0.0,
                            "raw_logprob": -0.01293118204921484
                        },
                        "topTokens": null,
                        "textRange": {"start": 0, "end": 1}
                    },
                    //...
                ]
            },
            "finishReason": {"reason": "endoftext"}
        }
    ]
}
```

### Anthropic Claude

#### Input

```json
{
    "prompt": "\n\nHuman:<prompt>\n\nAnswer:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
}
```

#### Output

```json
{
    "completion": "<output>",
    "stop_reason": "stop_sequence"
}
```

### Stability AI Stable Diffusion XL

#### Input

```json
{
    "text_prompts": [
        {"text": "this is where you place your input text"}
    ],
    "cfg_scale": 10,
    "seed": 0,
    "steps": 50
}
```

#### Output

```json
{ 
    "result": "success", 
    "artifacts": [
        {
            "seed": 123, 
            "base64": "<image in base64>",
            "finishReason": "SUCCESS"
        },
        //...
    ]
}
```

---

## Common inference parameter definitions

### Randomness and Diversity

Foundation models support the following parameters to control randomness and diversity in the 
response.

**Temperature** – Large language models use probability to construct the words in a sequence. For any 
given next word, there is a probability distribution of options for the next word in the sequence. When 
you set the temperature closer to zero, the model tends to select the higher-probability words. When 
you set the temperature further away from zero, the model may select a lower-probability word.

In technical terms, the temperature modulates the probability density function for the next tokens, 
implementing the temperature sampling technique. This parameter can deepen or flatten the density 
function curve. A lower value results in a steeper curve with more deterministic responses, and a higher 
value results in a flatter curve with more random responses.

**Top K** – Temperature defines the probability distribution of potential words, and Top K defines the cut 
off where the model no longer selects the words. For example, if K=50, the model selects from 50 of the 
most probable words that could be next in a given sequence. This reduces the probability that an unusual 
word gets selected next in a sequence.
In technical terms, Top K is the number of the highest-probability vocabulary tokens to keep for Top-
K-filtering - This limits the distribution of probable tokens, so the model chooses one of the highest-
probability tokens.

**Top P** – Top P defines a cut off based on the sum of probabilities of the potential choices. If you set Top 
P below 1.0, the model considers the most probable options and ignores less probable ones. Top P is 
similar to Top K, but instead of capping the number of choices, it caps choices based on the sum of their 
probabilities.
For the example prompt "I hear the hoof beats of ," you may want the model to provide "horses," 
"zebras" or "unicorns" as the next word. If you set the temperature to its maximum, without capping 
Top K or Top P, you increase the probability of getting unusual results such as "unicorns." If you set the 
temperature to 0, you increase the probability of "horses." If you set a high temperature and set Top K or 
Top P to the maximum, you increase the probability of "horses" or "zebras," and decrease the probability 
of "unicorns."

### Length

The following parameters control the length of the generated response.

**Response length** – Configures the minimum and maximum number of tokens to use in the generated 
response.

**Length penalty** – Length penalty optimizes the model to be more concise in its output by penalizing 
longer responses. Length penalty differs from response length as the response length is a hard cut off for 
the minimum or maximum response length.

In technical terms, the length penalty penalizes the model exponentially for lengthy responses. 0.0 
means no penalty. Set a value less than 0.0 for the model to generate longer sequences, or set a value 
greater than 0.0 for the model to produce shorter sequences.

### Repetitions

The following parameters help control repetition in the generated response.

**Repetition penalty (presence penalty)** – Prevents repetitions of the same words (tokens) in responses. 
1.0 means no penalty. Greater than 1.0 decreases repetition.

---

## Try out the models

With some theory out of the way, let's see the models in action! Run the cells below to see basic, synchronous example invocations for each model:

In [6]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

### Amazon Titan Express

In [7]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Command: Write me a blog about making strong business decisions as a leader.

Blog:
"""

In [8]:
try:
    
    body = json.dumps({"inputText": prompt_data})
    modelId = "amazon.titan-text-express-v1"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("results")[0].get("outputText"))

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

As a leader, making strong business decisions is crucial for the success of your organization. Here are some key principles to guide you in making informed and effective decisions:

Define Your Vision: Before making any decision, it's important to have a clear vision of what you want to achieve. This vision should be shared with your team and should guide all of your decision-making processes.

Gather Information: Gather as much information as possible about the decision you need to make. This includes data, market trends, customer feedback, and internal metrics. Use this information to inform your decision-making process.

Consider Alternatives:


### Anthropic Claude

In [9]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: Write me a blog about making strong business decisions as a leader.

Assistant:
"""

In [10]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:
    
    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

 Here is a draft blog post on making strong business decisions as a leader:

Making Strong Business Decisions as a Leader  

As a leader in business, one of the most important skills is being able to make strong, thoughtful decisions that will positively impact the company and employees. Here are some tips for making effective business decisions:

- Gather all relevant information. Don't make a decision without first doing thorough research and gathering facts, data, and perspectives from different sources. Make sure you understand the full context before deciding.

- Weigh pros and cons. Look at the potential benefits and risks of each option. Think critically about all possible outcomes, both positive and negative. Write out pros and cons lists if it helps visualize. 

- Consult experts and team members. Get input from people with expertise in the area as well as those who will be impacted by the decision. Different perspectives lead to stronger decisions.

- Take your time. Don't fe

### AI21 Jurassic

In [11]:
body = json.dumps({"prompt": prompt_data, "maxTokens": 200})
modelId = "ai21.j2-mid-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:
    
    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completions")[0].get("data").get("text"))

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

Making strong business decisions as a leader is an essential skill for any professional in a position of authority. These decisions can have far-reaching consequences, affecting not only the immediate outcomes but also the overall trajectory and success of the organization. Here are some key tips to keep in mind when making business decisions as a leader:

1. Understand the problem: Before making a decision, it's important to thoroughly understand the problem or opportunity at hand. This means taking the time to gather and analyze all available information, including market trends, customer insights, and financial data. Only by understanding the situation can you accurately identify the best solution and judge the potential consequences of different choices.
2. Consider multiple options: Instead of jumping to conclusions or making impulsive decisions, consider multiple options and take a balanced view of the situation. This can be accomplished by conducting a cost-benefit analysis or b

### Stability Stable Diffusion XL

In [12]:
prompt_data = "a fine image of an astronaut riding a horse on Mars"
body = json.dumps({
    "text_prompts": [{"text": prompt_data}],
    "cfg_scale": 10,
    "seed": 20,
    "steps": 50
})
modelId = "stability.stable-diffusion-xl"
accept = "application/json"
contentType = "application/json"

try:
    
    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body["result"])
    print(f'{response_body.get("artifacts")[0].get("base64")[0:80]}...')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

success
iVBORw0KGgoAAAANSUhEUgAAAgAAAAIACAIAAAB7GkOtAAABrGVYSWZNTQAqAAAACAAGAQAABAAAAAEA...


**Note:** The output is a [base64 encoded](https://docs.python.org/3/library/base64.html) string of the image data. You can use any image processing library (such as [Pillow](https://pillow.readthedocs.io/en/stable/)) to decode the image as in the example below:

```python
import base64
import io
from PIL import Image

base_64_img_str = response_body.get("artifacts")[0].get("base64")
image = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
```

## Generate streaming output

For large language models, it can take noticeable time to generate long output sequences. Rather than waiting for the entire response to be available, latency-sensitive applications may like to **stream** the response to users.

Run the code below to see how you can achieve this with Bedrock's `invoke_model_with_response_stream()` method - returning the response body in separate chunks.

In [13]:
from IPython.display import clear_output, display, display_markdown, Markdown

body = json.dumps({"inputText": prompt_data})
modelId = "amazon.titan-text-express-v1"  # (Change this, and the request body, to try different models)
accept = "application/json"
contentType = "application/json"

try:
    
    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    stream = response.get('body')
    output = []

    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                clear_output(wait=True)
                output.append(text)
                display_markdown(Markdown(''.join(output)))
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


This LLM does not have access to the Internet and cannot access any images. It has been trained to engage in conversation and generate responses to your questions.

### Anthropic Claude

In [14]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: Write me a blog about making strong business decisions as a leader.

Assistant:
"""

In [15]:
import botocore
from IPython.display import clear_output, display, display_markdown, Markdown

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  # (Change this to try different model versions)
accept = "application/json"
contentType = "application/json"

try:
    
    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    stream = response.get('body')
    output = []

    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['completion']
                clear_output(wait=True)
                output.append(text)
                display_markdown(Markdown(''.join(output)))
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

 Here is a draft blog post on making strong business decisions as a leader:

## How to Make Strong Business Decisions as a Leader

Being a leader means you will constantly be faced with important business decisions that could impact your company's future. While making choices is an inevitable part of any job, it's important as a leader to make the right decisions. Here are some tips for making wise, impactful business decisions:

**Look at the big picture.** When considering options, think about how each decision will affect your long-term goals and overall company strategy. Avoid getting bogged down in short-term wins if they compromise where you want the business to be in 5 or 10 years. 

**Gather all relevant data.** Do thorough research before committing to a path. Look at past performance metrics, industry trends, customer feedback, competitive analyses - anything that could provide useful context. Don't rely on assumptions or gut feelings alone.

**Involve the right people.** Bring in others who have valuable experience or stake in the outcome. Get input from employees, partners, advisors - groups impacted by or knowledgeable about the decision topic. Diverse perspectives help uncover aspects you may have overlooked.

**Consider risks and downsides.** Every choice has potential pros and cons. Carefully weigh what could go wrong as well as what could go right. Think through contingency plans in case things don't progress as hoped. Being aware of vulnerabilities allows for smarter risk mitigation.

**Trust your instincts - to an extent.** Valuable intuition comes from accumulated knowledge and experience. If initial analyses point one way but your gut says otherwise, reexamine assumptions. However, truly major decisions still require solid rationale; intuition alone isn't enough. 

**Be willing to change course if needed.** Committing to a path doesn't mean staying rigid if new information arises. Decisive leadership requires acting, then assessing outcomes and adapting as situations evolve. Don't be afraid to course-correct if monitoring suggests the initial decision isn't working out as planned.

Making strong choices is a key job duty of any business leader. By thoroughly researching options, considering various perspectives, properly managing risks, and maintaining flexibility, you can make the best possible decisions to propel your company forward.

## Generate embeddings

Use text embeddings to convert text into meaningful vector representations. You input a body of text 
and the output is a (1 x n) vector. You can use embedding vectors for a wide variety of applications. 
Bedrock currently offers Titan Embeddings for text embedding that supports text similarity (finding the 
semantic similarity between bodies of text) and text retrieval (such as search).

At the time of writing you can use `amazon.titan-embed-g1-text-02` as embedding model via the API. The input text size is 8192 tokens and the output vector length is 1536.

To use a text embeddings model, use the InvokeModel API operation or the Python SDK.
Use InvokeModel to retrieve the vector representation of the input text from the specified model.



#### Input

```json
{
    "inputText": "<text>"
}
```

#### Output

```json
{
    "embedding": []
}
```


Let's see how to generate embeddings of some text:

In [16]:
prompt_data = "Amazon Bedrock supports foundation models from industry-leading providers such as \
AI21 Labs, Anthropic, Stability AI, and Amazon. Choose the model that is best suited to achieving \
your unique goals."

In [17]:
body = json.dumps({"inputText": prompt_data})
modelId = "amazon.titan-embed-text-v1"  # (Change this to try different embedding models)
accept = "application/json"
contentType = "application/json"

try:
    
    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    embedding = response_body.get("embedding")
    print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

The embedding vector has 1536 values
[0.16601562, 0.23632812, 0.703125, '...', 0.26953125, -0.609375, -0.55078125]


## Next steps

In this notebook we showed some basic examples of invoking Amazon Bedrock models using the AWS Python SDK. You're now ready to explore the other labs to dive deeper on different use-cases and patterns.